# API ideas brainstorm

Quinn Marsh

The goal here is to allow any collaborators to see what ideas are or have been on the table

Entires are in reverse chronological order

In [ ]:
# Setup: Import libraries and define the Rosenbrock function
import functioneer as fn

# Rosenbrock function (known minimum of 0 at: x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a - x)**2 + b * (y - x**2)**2

In [ ]:
# 2025-06-25: API for Single and Multi-parameter Forks

# Best API for Single and Multi-parameter Forks (and Define)
# There are parallels between 'define' and 'fork' which both can have simple/single intpu BUT could also benifit from multiple being input in a single step
# As we think of a new API it'd be nice if we had parallel structure between the define and the fork

anal = fn.AnalysisModule(dict(a=1, b=100))

# Option 1: separate functions (current)
anal.add.define('a', 1)
anal.add.define('b', 100)
anal.add.fork('x', (0, 1, 2)) ## love the simplicity here, single param forks will be majority of use cases
anal.add.fork.multi(('x', 'y'), ((0, 1, 2), (0, 10, 20))) # don't like this compared to dict methods
anal.add.fork(param_id='x', value_set=(0, 1, 2)) ## love the simplicity here, single param forks will be majority of use cases
anal.add.fork.multi(param_ids=('x', 'y'), value_sets=((0, 1, 2), (0, 10, 20))) # don't like this compared to dict methods

# Option 2: always dict input 
# i like this because its consistent BUT we do loose the super simple format of anal.add.fork('x', (0, 1, 2))
anal.add.define({'a': 1, 'b': 100})
anal.add.fork({'x': (0, 1, 2)})
anal.add.fork({'x': (0, 1, 2), 'y': (0, 10, 20)}) # love the dict input in general because it pairs the number with its

# Option 3: allow both simple input from option 1 OR the dict from option 2
# this is not very explxit and would need dynamic args which could be really confusing but it would give both options
anal.add.fork('x', (0, 1, 2))
anal.add.fork({'x': (0, 1, 2)})
anal.add.fork({'x': (0, 1, 2), 'y': (0, 10, 20)})

# Option 4: seperate functions but with improved dict input
anal.add.fork('x', (0, 1, 2))
anal.add.fork.dict({'x': (0, 1, 2)}) # or different function name
anal.add.fork.dict({'x': (0, 1, 2), 'y': (0, 10, 20)})

# Option 5: separate arg for dicts and all args optional
anal.add.fork(param_id='x', value_set=(0, 1, 2))
anal.add.fork(dict={'x': (0, 1, 2)}) # or different arg name
anal.add.fork(dict={'x': (0, 1, 2), 'y': (0, 10, 20)})

# Option 6: combined fork and define
anal.add({'a': {'value': 1}, 'b': {'value': 100}, 'x': {'fork': (0, 1, 2)}, 'y': {'fork': (0, 100, 200)}}) # this seems so clunky tbh but maybe it can inspire a good idea

# Option 7: 
# Open to anythin else

anal.add.execute(func=rosenbrock)
results = anal.run()
print('\nTest Case 4 Output:')
print(results['df'].drop(columns='datetime'))

In [ ]:
# Initial API Ideas

## Version 1: Add steps as Objects ---------------------------------------------------------------------------------------
# PROs: directly shows use whats going on under the hood
# CONs: user must know about and import functioneer objects
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2))) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add(fn.Fork('b', value_sets=(0, 100, 200))) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

results = anal.run()


## Version 2: Add steps through the 'add' namespace ---------------------------------------------------------------------------------------
# PROs: relatively short
# CONs:
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add.define('x', 0)
anal.add.define('y', 0)

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

results = anal.run()


## Version 3: Add steps directly to Analysis ---------------------------------------------------------------------------------------
# PROs: shorter
# CONs: does not correctly convey to the user that they are 'adding' or queuing up steps, in the future there may be more non-adding functions like run that need to be distinct from add
anal = fn.AnalysisModule()

# Define analysis sequence
anal.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.define('x', 0)
anal.define('y', 0)

anal.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()


## Version 4: Declarative list of Objects ---------------------------------------------------------------------------------------
# PROs: declarative
# CONs: does not allow for definting of functions between adding steps which i think is important to functioneer being able to conform to any persons mental flow
anal = fn.AnalysisModule()

# Define the analysis as a configuration
anal = fn.AnalysisModule([
    fn.Fork('a', value_sets=(1, 2)),
    fn.Fork('b', value_sets=(0, 100, 200)),
    fn.Define('x', 0),
    fn.Define('y', 0),
    fn.Execute(method='default', func=rosenbrock),
    fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')),
])

# Run the analysis
results = anal.run()